<a href="https://colab.research.google.com/github/lisahligono/GeoHUM/blob/main/Population_estimation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This is a sample code in estimating population in Venezuela using deep learning algorithms. (Author: Lisah Khadiala Ligono) GeoHUM class assignment

**Step 1**:
To obtain a high-resolution satellite image of an area in Google Colab, you will need to use the **urllib** library to download the image from a remote server. 
This code will download the image from the specified URL and save it to a local file called satellite_image.jpg. It will then load the image into memory as a NumPy array using the cv2 library.

In [ ]:
import urllib.request

# URL of the satellite image.
image_url = "https://www.example.com/satellite_image.jpg"

# Download the image and save it to a local file.
urllib.request.urlretrieve(image_url, "satellite_image.jpg")

# Load the image into memory as a NumPy array.
image = cv2.imread("satellite_image.jpg")

In [ ]:
#Installations
!apt install gdal-bin python-gdal python3-gdal 
!pip install rasterio
!apt install python3-rtree 
!pip install git+git://github.com/geopandas/geopandas.git
!pip install descartes
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt
import rasterio as rio
from rasterio.plot import show

fpath = 'http://landsat-pds.s3.amazonaws.com/c1/L8/003/065/LC08_L1TP_003065_20190925_20191017_01_T1/LC08_L1TP_003065_20190925_20191017_01_T1_B4.TIF'

def rasterio_open(f):
    return rio.open(f)

fig, ax = plt.subplots(1, figsize=(12, 10))
show(src_image, ax=ax)
plt.show()



**Step 2**: resize the image to 256x256 pixels, crop it to the area of interest, and convert it to a tensor using the torch library.

In [ ]:
import cv2
import torch

def preprocess_image(image):
  # Resize the image to a smaller size, such as 256x256 pixels.
  image = cv2.resize(image, (256, 256))
  
  # Crop the image to the area of interest.
  image = image[50:200, 50:200]
  
  # Convert the image to a tensor.
  image = torch.from_numpy(image).float()
  
  return image

# Preprocess the satellite image.
image = preprocess_image(image)

**Step 3:** This code will use the annotation_tool library to annotate each image in the images list. For each image, it will draw bounding boxes around the buildings and label them with the class "building". The bounding boxes and labels are then added to the dataset list.

In [ ]:
import annotation_tool

def create_ground_truth_dataset(images):
  dataset = []
  
  for image in images:
    # Load the preprocessed image into the annotation tool.
    # Use the annotation tool to draw bounding boxes around each building in the image.
    # Label each bounding box with the class "building".
    annotated_image = annotation_tool.annotate(image)
    
    # Extract the bounding boxes and labels from the annotated image.
    boxes = annotated_image.get_boxes()
    labels = annotated_image.get_labels()
    
    # Add the boxes and labels to the dataset.
    dataset.append((boxes, labels))
  
  return dataset

# Create the ground truth dataset.
dataset = create_ground_truth_dataset(images)

**Step 4**:This code will use the train_test_split function from the sklearn library to split the dataset into training and validation sets. The test_size argument specifies the proportion of the dataset to include in the validation set. In this case, the validation set will contain 20% of the data and the training set will contain the remaining 80%.


In [ ]:
from sklearn.model_selection import train_test_split

# Split the annotated dataset into training and validation sets.
train_set, val_set = train_test_split(dataset, test_size=0.2)

**Step 5**:This code defines a CNN called BuildingDetector that consists of three convolutional layers, three pooling layers, and two fully connected layers. The input to the network is a 3-channel image, which is passed through the convolutional layers and pooling layers before being flattened and passed through the fully connected layers. The output of the network is a single scalar value representing the probability that a building is present in the image.

In [ ]:
import torch.nn as nn
import torch.nn.functional as F

class BuildingDetector(nn.Module):
  def __init__(self):
    super(BuildingDetector, self).__init__()
    
    # Define the convolutional layers.
    self.conv1 = nn.Conv2d(3, 16, kernel_size=3, padding=1)
    self.conv2 = nn.Conv2d(16, 32, kernel_size=3, padding=1)
    self.conv3 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
    
    # Define the pooling layers.
    self.pool = nn.MaxPool2d(2, 2)
    
    # Define the fully connected layers.
    self.fc1 = nn.Linear(64 * 64 * 64, 512)
    self.fc2 = nn.Linear(512, 1)
    
  def forward(self, x):
    # Pass the input through the convolutional layers and pooling layers.
    x = self.pool(F.relu(self.conv1(x)))
    x = self.pool(F.relu(self.conv2(x)))
    x = self.pool(F.relu(self.conv3(x)))
    
    # Flatten the output.
    x = x.view(-1, 64 * 64 * 64)
    
    # Pass the output through the fully connected layers.
    x = F.relu(self.fc1(x))
    x = self.fc2(x)
    
    return x

# Create an instance of the BuildingDetector CNN.
model = BuildingDetector()

**Step 6**: This code selects the binary_cross_entropy_with_logits loss function from the torch.nn.functional library and the Adam optimizer from the torch library. The Adam optimizer is a popular choice for training deep learning models and is generally easy to use.

In [ ]:
import torch.nn.functional as F

# Select an appropriate loss function.
loss_fn = F.binary_cross_entropy_with_logits

# Select an appropriate optimizer.
optimizer = torch.optim.Adam(model.parameters())

**Step 7**: This code trains the model on the training set for 10 epochs. For each epoch, it loops over the training set and performs a forward pass through the CNN, computes the loss, and performs a backward pass to compute the gradients. It then updates the weights of the model using the optimizer.

In [ ]:
import torch

# Set the number of epochs.
num_epochs = 10

# Set the model to training mode.
model.train()

# Train the model on the training set for several epochs.
for epoch in range(num_epochs):
  # Loop over the training set.
  for images, labels in train_set:
    # Preprocess the images.
    images = preprocess_image(images)
    
    # Clear the gradients.
    optimizer.zero_grad()
    
    # Forward pass.
    outputs = model(images)
    
    # Compute the loss.
    loss = loss_fn(outputs, labels)
    
    # Backward pass.
    loss.backward()
    
    # Update the weights.
    optimizer.step()
  
  # Print the progress.
  print("Epoch %d/%d" % (epoch+1, num_epochs))

**Step 8**:This code evaluates the model on the validation set and computes the average loss and accuracy. It sets the model to evaluation mode before performing the forward pass and computes the loss using the specified loss function. It then sets the model back to training mode.

In [ ]:
import torch

def evaluate(model, val_set, loss_fn):
  # Set the model to evaluation mode.
  model.eval()
  
  # Initialize variables to store the total loss and number of correct predictions.
  total_loss = 0
  num_correct = 0
  
  # Loop over the validation set.
  for images, labels in val_set:
    # Preprocess the images.
    images = preprocess_image(images)
    
    # Forward pass.
    outputs = model(images)
    
    # Compute the loss.
    loss = loss_fn(outputs, labels)
    
    # Update the total loss and number of correct predictions.
    total_loss += loss.item()
    num_correct += (torch.sigmoid(outputs) >= 0.5).squeeze().sum().item()
  
  # Compute the average loss and accuracy.
  avg_loss = total_loss / len(val_set)
  avg_acc = num_correct / len(val_set)
  
  # Set the model back to training mode.
  model.train()
  
  return avg_loss, avg_acc

# Evaluate the model on the validation set.
avg_loss, avg_acc = evaluate(model, val_set, loss_fn)

# Print the results.
print("Average loss: %.4f" % avg_loss)
print("Average accuracy: %.4f" % avg_acc)

**Step 9**:This code loads a new satellite image and passes it through the trained CNN to predict the probability that a building is present in the image. The probability is computed using the sigmoid function from the torch library.

In [ ]:
import cv2
import torch

def predict(model, image):
  # Preprocess the image.
  image = preprocess_image(image)
  
  # Forward pass.
  output = model(image)
  
  # Convert the output to a probability.
  prob = torch.sigmoid(output).item()
  
  return prob

# Load a new satellite image.
image = cv2.imread("new_image.jpg")

# Predict the presence of buildings in the image.
prob = predict(model, image)

print("Probability of building: %.4f" % prob)

**Step 10**:This code uses the trained CNN to predict the number of buildings in the satellite image and then uses the predict method of the regressor object to predict the population based on the number of buildings. The regressor object could be a trained linear regression model, for example.



In [ ]:
import cv2
import numpy as np
import torch

def estimate_population(model, image, regressor):
  # Predict the number of buildings in the image.
  prob = predict(model, image)
  num_buildings = prob * 1000
  
  # Use the regressor to predict the population based on the number of buildings.
  population = regressor.predict(num_buildings)
  
  return population

# Load a new satellite image.
image = cv2.imread("new_image.jpg")

# Estimate the population of the area in the image.
population = estimate_population(model, image, regressor)

print("Estimated population: %d" % population)
